In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
from sqlalchemy import create_engine
import pandas as pd
from functions.utils import add_matchup_probabilities, create_advanced_features, get_team_stats
import numpy as np
import pandas as pd

##  Sample from postgre

In [3]:
username = 'rodrigo'
host = 'localhost'           
port = '5432'               
database = 'futmondo_full_players_info'

In [4]:
engine = create_engine(f'postgresql+psycopg2://{username}@{host}:{port}/{database}')

In [5]:
query = "SELECT * FROM player_points"
df = pd.read_sql(query, engine)
df.shape

(2064, 17)

In [6]:
df[df['name']=='Toljan']

,player_id,name,role,round,team_id,home_average,away_average,overall_average,current_price,matches_played,rating,match_minus_1,match_minus_2,last_2_average,target_points,unique_id,team
41,51b9e593e401a15f2c00019f,Toljan,defensa,14,504e581e4d8bec9a670000cf,3.166667,5.666667,4.666667,13710597,15,4,6,3,4.5,5.0,2900834f0f99588003b3c29c6b9557c6e3d7c5d5b21891...,Levante
558,51b9e593e401a15f2c00019f,Toljan,defensa,15,504e581e4d8bec9a670000cf,3.166667,5.666667,4.666667,13710597,15,4,5,6,5.5,0.0,fe4c2a0ab7b016edd04c481c8b66669ef85a92ec3f53b4...,Levante
1074,51b9e593e401a15f2c00019f,Toljan,defensa,16,504e581e4d8bec9a670000cf,3.166667,5.666667,4.666667,13710597,15,4,0,5,2.5,0.0,5e9c0568825a44770439292a375e5e6ce55a4b8aba4f8f...,Levante
1590,51b9e593e401a15f2c00019f,Toljan,defensa,17,504e581e4d8bec9a670000cf,3.166667,5.666667,4.666667,13710597,15,4,0,0,0.0,NaN,f234d9de45ba07470711bcd28617db8fd15b2a848ce625...,Levante


### 1. Create new rows for next rounds

This is not necessary with the last database

In [7]:
# # Get the latest round for each player
# latest_rounds = df.groupby('player_id')['round'].max().reset_index()
# latest_rounds.columns = ['player_id', 'latest_round']

# # Create next round data
# next_round_data = []

# for _, row in latest_rounds.iterrows():
#     player_id = row['player_id']
#     latest_round = row['latest_round']
    
#     # Get player's latest record
#     player_latest = df[(df['player_id'] == player_id) & (df['round'] == latest_round)].iloc[0]
    
#     # Get last 3 rounds for new average
#     player_last_3 = df[(df['player_id'] == player_id) & (df['round'] > latest_round - 3)].sort_values('round')
    
#     # Calculate new last_3_average
#     if len(player_last_3) >= 3:
#         new_last_3_avg = player_last_3.tail(3)['target_points'].mean()
#     else:
#         new_last_3_avg = player_last_3['target_points'].mean()
    
#     # Create new row
#     new_row = player_latest.copy()
#     new_row['round'] = latest_round + 1
#     new_row['last_3_average'] = new_last_3_avg
#     new_row['match_minus_3'] = player_latest['match_minus_2']
#     new_row['match_minus_2'] = player_latest['match_minus_1']
#     new_row['match_minus_1'] = player_latest['target_points']
#     new_row['is_home_target'] = not player_latest['is_home_target']  # Flip home/away
#     new_row['target_points'] = None
    
#     next_round_data.append(new_row)

# # Create DataFrame and combine
# next_round_df = pd.DataFrame(next_round_data)
# df_complete = pd.concat([df, next_round_df], ignore_index=True)

Now I have updated and I have la liga matches with the next round probabilities

In [7]:
username = 'rodrigo'
host = 'localhost'           
port = '5432'               
database = 'futmondo_full_players_info'

In [8]:
engine = create_engine(f'postgresql+psycopg2://{username}@{host}:{port}/{database}')

In [9]:
query = "SELECT * FROM la_liga_matches"
df_la_liga = pd.read_sql(query, engine)
df_la_liga.shape

(190, 17)

In [10]:
# df_la_liga_15 = df_la_liga[df_la_liga['Round'] != 15]

### 2. Create new round values

In [11]:
df_liga_next=pd.read_csv('../data/la_liga_next_rounds.csv')

In [12]:
# Load your datasets
historical_df = df_la_liga
upcoming_df = df_liga_next

# Clean team names (strip whitespace)
historical_df['HomeTeam'] = historical_df['HomeTeam'].str.strip()
historical_df['AwayTeam'] = historical_df['AwayTeam'].str.strip()
upcoming_df['Home Team'] = upcoming_df['Home Team'].str.strip()
upcoming_df['Away Team'] = upcoming_df['Away Team'].str.strip()

# Standardize team names in upcoming dataset to match historical
name_mapping = {
    'Real Sociedad': 'Sociedad',
    'Atlético Madrid': 'Ath Madrid',
    'Celta Vigo': 'Celta',
    'Alavés': 'Alaves',
    'Rayo Vallecano': 'Vallecano',
    'Real Betis': 'Betis',
    'Real Oviedo': 'Oviedo',
    'Athletic Bilbao': 'Ath Bilbao',
    'Athletic Club': 'Ath Bilbao',
    'Espanyol': 'Espanol'
}

upcoming_df['Home Team'] = upcoming_df['Home Team'].replace(name_mapping)
upcoming_df['Away Team'] = upcoming_df['Away Team'].replace(name_mapping)

# Get unique teams from both datasets
historical_teams = set(historical_df['HomeTeam'].unique()) | set(historical_df['AwayTeam'].unique())
upcoming_teams = set(upcoming_df['Home Team'].unique()) | set(upcoming_df['Away Team'].unique())

# Check for team name mismatches
missing_teams = upcoming_teams - historical_teams
if missing_teams:
    print(f"Warning: These teams in upcoming matches not found in historical data: {missing_teams}")

team_stats = get_team_stats(historical_df)

# Create predictions for upcoming matches
predictions = []

for idx, row in upcoming_df.iterrows():
    home_team = row['Home Team']
    away_team = row['Away Team']
    round_num = row['Round']
    
    # Get team stats (use league average if team not found)
    league_avg_home_win = historical_df['Prob_Home_Norm'].mean()
    league_avg_draw = historical_df['Prob_Draw_Norm'].mean()
    league_avg_away_win = historical_df['Prob_Away_Norm'].mean()
    
    # Get home team's home performance
    if home_team in team_stats and 'home_win_prob' in team_stats[home_team]:
        home_win_strength = team_stats[home_team]['home_win_prob']
        home_draw_strength = team_stats[home_team]['home_draw_prob']
    else:
        home_win_strength = league_avg_home_win
        home_draw_strength = league_avg_draw
    
    # Get away team's away performance
    if away_team in team_stats and 'away_win_prob' in team_stats[away_team]:
        away_win_strength = team_stats[away_team]['away_win_prob']
        away_draw_strength = team_stats[away_team]['away_draw_prob']
    else:
        away_win_strength = league_avg_away_win
        away_draw_strength = league_avg_draw
    
    # Simple average of team strengths
    prob_home = (home_win_strength + (1 - away_win_strength)) / 2
    prob_away = (away_win_strength + (1 - home_win_strength)) / 2
    prob_draw = (home_draw_strength + away_draw_strength) / 2
    
    # Normalize probabilities to sum to 1
    total = prob_home + prob_draw + prob_away
    prob_home_norm = prob_home / total
    prob_draw_norm = prob_draw / total
    prob_away_norm = prob_away / total
    
    # Convert to odds
    avg_h = 1 / prob_home_norm if prob_home_norm > 0 else 999
    avg_d = 1 / prob_draw_norm if prob_draw_norm > 0 else 999
    avg_a = 1 / prob_away_norm if prob_away_norm > 0 else 999
    
    predictions.append({
        'Date': None,  # To be filled
        'HomeTeam': home_team,
        'AwayTeam': away_team,
        'FTHG': None,
        'FTAG': None,
        'FTR': None,
        'AvgH': round(avg_h, 2),
        'AvgD': round(avg_d, 2),
        'AvgA': round(avg_a, 2),
        'Prob_Home': round(prob_home, 6),
        'Prob_Draw': round(prob_draw, 6),
        'Prob_Away': round(prob_away, 6),
        'Total': round(total, 6),
        'Prob_Home_Norm': round(prob_home_norm, 6),
        'Prob_Draw_Norm': round(prob_draw_norm, 6),
        'Prob_Away_Norm': round(prob_away_norm, 6),
        'Round': round_num
    })

# Create predictions dataframe
predictions_df = pd.DataFrame(predictions)

# Combine with historical data
updated_df = pd.concat([historical_df, predictions_df], ignore_index=True)

/var/folders/r5/bx2jhb6n64n4zm91gw712nr00000gn/T/ipykernel_8302/1519126780.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([historical_df, predictions_df], ignore_index=True)


In [13]:
# updated_df.to_sql('la_liga_matches', engine, if_exists='replace', index=False)  Already updated

### 3. Combine and enrich the data

In [14]:
df_complete=df.copy()

In [15]:
df_complete.head(2)

,player_id,name,role,round,team_id,home_average,away_average,overall_average,current_price,matches_played,rating,match_minus_1,match_minus_2,last_2_average,target_points,unique_id,team
0,576cb7e161f452a36e9181f4,Cervi,delantero,14,504e581e4d8bec9a670000d9,0.000000,0.000,0.0,1000000,0,1,0,0,0.0,0.0,f1b7e6b357d9a9ba0bf1a137b26e3e815ae39a5a0fa99b...,Celta
1,520d1ce5a776cc826b000026,Lucas Torró,centrocampista,14,504e581e4d8bec9a670000d1,4.428571,3.625,4.0,9153244,15,3,3,0,1.5,6.0,1d44af795280ebe969326c7bfb173a21effd044e406c2b...,Osasuna


In [16]:
df_complete.shape

(2064, 17)

In [18]:
# df_complete.head(2)

In [19]:
# name_mapping = {
#     'Alavés': 'Alaves',
#     'Atlético de Madrid': 'Ath Madrid',
#     'Celta de Vigo': 'Celta',
#     'Alavés': 'Alaves',
#     'Rayo Vallecano': 'Vallecano',
#     'Espanyol': 'Espanol',
#     'Athletic de Bilbao': 'Ath Bilbao',
#     'Real Sociedad': 'Sociedad'}

# df_complete['team'] = df_complete['team'].replace(name_mapping)

In [17]:
df_points=df_complete.copy()
df_matches=updated_df.copy()

In [18]:
df_add=add_matchup_probabilities(df_points, df_matches)

In [19]:
df_add.shape

(2064, 22)

In [20]:
df_add.columns

Index(['player_id', 'name', 'role', 'round', 'team_id', 'home_average',
       'away_average', 'overall_average', 'current_price', 'matches_played',
       'rating', 'match_minus_1', 'match_minus_2', 'last_2_average',
       'target_points', 'unique_id', 'team', 'matchup_prob_win',
       'matchup_prob_draw', 'matchup_prob_loss', 'is_home', 'opponent'],
      dtype='object')

In [21]:
df_enriched=create_advanced_features(df_add)

/Users/rodrigo/football-data-analytics/futmondo_points_predict/functions/utils.py:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0.0)
/Users/rodrigo/football-data-analytics/futmondo_points_predict/functions/utils.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0.0)
/Users/rodrigo/football-data-analytics/futmondo_points_predict/functions/utils.py:109: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-i

In [22]:
df_enriched.columns

Index(['player_id', 'name', 'role', 'round', 'team_id', 'home_average',
       'away_average', 'overall_average', 'current_price', 'matches_played',
       'rating', 'match_minus_1', 'match_minus_2', 'last_2_average',
       'target_points', 'unique_id', 'team', 'matchup_prob_win',
       'matchup_prob_draw', 'matchup_prob_loss', 'is_home', 'opponent',
       'form_trend', 'home_away_diff', 'price_per_point', 'price_efficiency',
       'recent_momentum', 'home_form_interaction', 'away_form_interaction',
       'location_adjusted_average', 'matchup_strength',
       'team_expected_performance', 'delantero_matchup_bonus',
       'centrocampista_matchup_bonus', 'defensa_matchup_bonus',
       'portero_matchup_bonus', 'home_matchup_boost', 'difficult_matchup',
       'easy_matchup'],
      dtype='object')

In [26]:
# df_koke=df_enriched[df_enriched['name']=='Koke']

In [27]:
# df_koke[['player_id', 'name', 'role', 'round', 'home_average', 'away_average',
#        'overall_average', 'current_price', 'matches_played', 'rating',
#        'team_id', 'match_minus_1', 'match_minus_2', 'last_2_average',
#        'target_points', 'unique_id', 'team']]

In [28]:
# df_koke=df_koke.dropna()

### 4. Final cleaning of data

In [30]:
# df_enriched = df_enriched.dropna()

In [27]:
df_enriched.shape

(2064, 39)

In [28]:
df_enriched.head(2)

,player_id,name,role,round,team_id,home_average,away_average,overall_average,current_price,matches_played,...,location_adjusted_average,matchup_strength,team_expected_performance,delantero_matchup_bonus,centrocampista_matchup_bonus,defensa_matchup_bonus,portero_matchup_bonus,home_matchup_boost,difficult_matchup,easy_matchup
0,576cb7e161f452a36e9181f4,Cervi,delantero,14,504e581e4d8bec9a670000d9,0.000000,0.000,0.0,1000000,0,...,0.0,0.222378,1.696362,0.947967,0.000000,0.0,0.0,0.236992,0,0
1,520d1ce5a776cc826b000026,Lucas Torró,centrocampista,14,504e581e4d8bec9a670000d1,4.428571,3.625,4.0,9153244,15,...,3.625,-0.117237,1.168529,0.000000,0.342919,0.0,0.0,0.0,0,0


In [29]:
def clean_value(x):
    if isinstance(x, (np.floating, np.integer)):
        return x.item()
    if isinstance(x, (np.bool_)):
        return bool(x)
    return x

df_enriched = df_enriched.applymap(clean_value)

/var/folders/r5/bx2jhb6n64n4zm91gw712nr00000gn/T/ipykernel_8302/2151177701.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_enriched = df_enriched.applymap(clean_value)


In [30]:
df_enriched.shape

(2064, 39)

In [31]:
df_enriched.to_csv('../data/final_dataset/futmondo_final_dataset.csv', index=False)

In [32]:
df_enriched.to_sql('full_training_data', engine, if_exists='replace', index=False)

388

In [37]:
# import os

# file_path = "../data/model/fantasy_model_complete.pkl"   # change this

# size_bytes = os.path.getsize(file_path)
# size_mb = size_bytes / (1024 * 1024)

# print(f"Size: {size_mb:.2f} MB")

In [ ]:
# model = joblib.load('fantasy_points_model.pkl')
# scaler = joblib.load('feature_scaler.pkl')
# feature_columns = joblib.load('feature_columns.pkl')